# How Random Forest Hyperparameters Shape Decision Boundaries
### Breast Cancer Wisconsin (Diagnostic) — Full tutorial notebook

This notebook contains a complete pipeline you can run in Google Colab or Jupyter:
- Load the **Breast Cancer** dataset (built-in sklearn version)
- EDA (brief)
- Random Forest feature importance and selecting top features
- Decision boundaries using top-2 raw features
- PCA (2 components) and decision boundary
- NCA (2 components) and decision boundary
- Compare methods and select the best
- Study individual hyperparameters (max_depth, n_estimators, max_features)
- GridSearchCV tuning and interpretation plots

Each code cell includes student-friendly comments and short textual explanation for key points and keywords.

Run the notebook from top to bottom. If using Colab, upload a copy of this notebook and run cells.


In [ ]:
# Imports and settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.inspection import permutation_importance

%matplotlib inline
print('Libraries loaded.')


## 1. Load data and quick overview

**Keywords:** dataset, features, target, shape

We use `sklearn.datasets.load_breast_cancer` for a clean copy of the Breast Cancer dataset.


In [ ]:
# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

print('Feature count:', X.shape[1])
print('Sample count:', X.shape[0])
print('Target names:', target_names)

# Put into a DataFrame for easy EDA
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df.head()


## 2. Brief EDA (exploratory data analysis)

Show distributions of a few top features and a correlation heatmap. **Keep EDA compact** for the report.


In [ ]:
# Simple distribution plots for three informative features
sample_features = ['mean radius', 'mean texture', 'mean perimeter']
for feat in sample_features:
    plt.figure()
    df.boxplot(column=feat, by='target')
    plt.title(f'{feat} by class (0=benign,1=malignant)')
    plt.suptitle('')
    plt.xlabel('Class')
    plt.ylabel(feat)
    plt.show()

# Correlation heatmap (small) - compute and plot
corr = df.drop(columns=['target']).corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest')
plt.title('Feature correlation matrix (visual)')
plt.colorbar()
plt.xticks(range(len(feature_names)), feature_names, rotation=90)
plt.yticks(range(len(feature_names)), feature_names)
plt.tight_layout()
plt.show()


## 3. Random Forest Feature Importance (Gini) and Permutation Importance

**Keywords:** feature importance, Gini importance, permutation importance, interpretability

We train a Random Forest on the full feature set and extract feature importances. Then compute permutation importance for more robust ranking.


In [ ]:
# Train/test split (full features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Train a baseline Random Forest
rf_full = RandomForestClassifier(n_estimators=200, random_state=42)
rf_full.fit(X_train, y_train)

# Gini-based feature importance
gini_importances = rf_full.feature_importances_
gini_idx = np.argsort(gini_importances)[::-1]
print('Top 10 features by Gini importance:')
for i in gini_idx[:10]:
    print(f"{feature_names[i]}: {gini_importances[i]:.4f}")

# Plot Gini importances
plt.figure(figsize=(8,6))
plt.barh(range(10), gini_importances[gini_idx[:10]][::-1])
plt.yticks(range(10), [feature_names[i] for i in gini_idx[:10]][::-1])
plt.xlabel('Gini importance')
plt.title('Top 10 features by Gini importance')
plt.tight_layout()
plt.show()

# Permutation importance (model-agnostic)
perm = permutation_importance(rf_full, X_test, y_test, n_repeats=20, random_state=42, n_jobs=-1)
perm_idx = np.argsort(perm.importances_mean)[::-1]
print('\nTop 10 features by permutation importance:')
for i in perm_idx[:10]:
    print(f"{feature_names[i]}: mean imp={perm.importances_mean[i]:.4f}, std={perm.importances_std[i]:.4f}")

# Plot permutation importances
plt.figure(figsize=(8,6))
plt.barh(range(10), perm.importances_mean[perm_idx[:10]][::-1])
plt.yticks(range(10), [feature_names[i] for i in perm_idx[:10]][::-1])
plt.xlabel('Permutation importance (mean)')
plt.title('Top 10 features by Permutation importance')
plt.tight_layout()
plt.show()


## 4. Pick top 2 original features (from permutation importance) and plot decision boundary

We use the top two features from permutation importance because they better reflect model performance.


In [ ]:
# Select top 2 features according to permutation importance
top2 = perm_idx[:2]
top2_names = [feature_names[i] for i in top2]
print('Top 2 features chosen for 2D plots:', top2_names)

# Extract 2D data
X2 = X[:, top2]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.3, random_state=42, stratify=y)

def plot_decision_boundary(model, X_plot, y_plot, title='Decision boundary'):
    # Grid for plotting
    x_min, x_max = X_plot[:, 0].min() - 1, X_plot[:, 0].max() + 1
    y_min, y_max = X_plot[:, 1].min() - 1, X_plot[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X_plot[:,0], X_plot[:,1], c=y_plot, edgecolor='k')
    plt.xlabel(top2_names[0])
    plt.ylabel(top2_names[1])
    plt.title(title)
    plt.show()

# Train RF on top2 and plot
rf_top2 = RandomForestClassifier(n_estimators=200, random_state=42)
rf_top2.fit(X2_train, y2_train)
print('Accuracy on test (top2 raw features):', accuracy_score(y2_test, rf_top2.predict(X2_test)))
plot_decision_boundary(rf_top2, X2_train, y2_train, title='Decision boundary (Top2 raw features)')


## 5. PCA (2 components) — transform, train RF, plot boundary

PCA is unsupervised and finds directions of maximum variance. We scale before PCA.


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, test_size=0.3, random_state=42, stratify=y)

rf_pca = RandomForestClassifier(n_estimators=200, random_state=42)
rf_pca.fit(X_pca_train, y_pca_train)
print('Accuracy on test (PCA 2 components):', accuracy_score(y_pca_test, rf_pca.predict(X_pca_test)))
plot_decision_boundary(rf_pca, X_pca_train, y_pca_train, title='Decision boundary (PCA 2 components)')


## 6. NCA (2 components) — supervised projection, train RF, plot boundary

NCA (Neighborhood Components Analysis) learns a projection that optimises nearest-neighbour classification. It is supervised and often produces better class separation than PCA for classification tasks.


In [ ]:
# NCA requires scaled data
scaler2 = StandardScaler()
X_scaled2 = scaler2.fit_transform(X)
nca = NeighborhoodComponentsAnalysis(n_components=2, random_state=42)
X_nca = nca.fit_transform(X_scaled2, y)
X_nca_train, X_nca_test, y_nca_train, y_nca_test = train_test_split(X_nca, y, test_size=0.3, random_state=42, stratify=y)

rf_nca = RandomForestClassifier(n_estimators=200, random_state=42)
rf_nca.fit(X_nca_train, y_nca_train)
print('Accuracy on test (NCA 2 components):', accuracy_score(y_nca_test, rf_nca.predict(X_nca_test)))
plot_decision_boundary(rf_nca, X_nca_train, y_nca_train, title='Decision boundary (NCA 2 components)')


## 7. Compare methods and choose best

Compare test accuracies and visual separation; choose the best for the final model.


In [ ]:
acc_raw = accuracy_score(y2_test, rf_top2.predict(X2_test))
acc_pca = accuracy_score(y_pca_test, rf_pca.predict(X_pca_test))
acc_nca = accuracy_score(y_nca_test, rf_nca.predict(X_nca_test))
print('Summary accuracies:')
print('Raw top2:', acc_raw)
print('PCA 2 comp:', acc_pca)
print('NCA 2 comp:', acc_nca)

methods = {'Raw':(acc_raw, 'raw'), 'PCA':(acc_pca, 'pca'), 'NCA':(acc_nca, 'nca')}
best = max(methods.items(), key=lambda x: x[1][0])
print('\nBest method:', best[0])

# Set final training/test accordingly
if best[0] == 'Raw':
    X_train_final, X_test_final = X2_train, X2_test
    y_train_final, y_test_final = y2_train, y2_test
elif best[0] == 'PCA':
    X_train_final, X_test_final = X_pca_train, X_pca_test
    y_train_final, y_test_final = y_pca_train, y_pca_test
else:
    X_train_final, X_test_final = X_nca_train, X_nca_test
    y_train_final, y_test_final = y_nca_train, y_nca_test

print('Chosen final feature set and shapes: ', X_train_final.shape, X_test_final.shape)


## 8. Study individual hyperparameters effects

We vary one hyperparameter at a time and plot accuracy vs value. This shows how each parameter affects performance.


In [ ]:
from sklearn.metrics import accuracy_score
param_results = {}

# max_depth
depth_values = [2, 4, 6, 8, None]
acc_depth = []
for d in depth_values:
    m = RandomForestClassifier(n_estimators=100, max_depth=d, random_state=42)
    m.fit(X_train_final, y_train_final)
    acc_depth.append(accuracy_score(y_test_final, m.predict(X_test_final)))
param_results['max_depth'] = (depth_values, acc_depth)

# n_estimators
tree_values = [10, 50, 100, 200, 300]
acc_trees = []
for n in tree_values:
    m = RandomForestClassifier(n_estimators=n, random_state=42)
    m.fit(X_train_final, y_train_final)
    acc_trees.append(accuracy_score(y_test_final, m.predict(X_test_final)))
param_results['n_estimators'] = (tree_values, acc_trees)

# max_features
feat_values = [1, 'sqrt', 'log2']
acc_feat = []
for f in feat_values:
    m = RandomForestClassifier(max_features=f, random_state=42)
    m.fit(X_train_final, y_train_final)
    acc_feat.append(accuracy_score(y_test_final, m.predict(X_test_final)))
param_results['max_features'] = (feat_values, acc_feat)

# Plot results
for name, (vals, accs) in param_results.items():
    plt.figure()
    # Convert vals to strings for plotting if they are not numeric
    plt.plot(list(range(len(vals))), accs, marker='o')
    plt.xticks(list(range(len(vals))), [str(v) for v in vals])
    plt.xlabel(name)
    plt.ylabel('Accuracy')
    plt.title(f'Effect of {name} on accuracy')
    plt.grid(True)
    plt.show()


## 9. Hyperparameter tuning: GridSearchCV

We run GridSearchCV on a sensible grid and inspect the best parameters and CV scores.


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 8, None],
    'max_features': [1, 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_train_final, y_train_final)
print('Best CV score:', grid.best_score_)
print('Best params:', grid.best_params_)

# Simple plot of mean test scores (sorted)
results = grid.cv_results_
means = results['mean_test_score']
stds = results['std_test_score']
plt.figure(figsize=(10,4))
plt.plot(means, marker='o')
plt.title('GridSearchCV mean test scores (all combinations)')
plt.xlabel('Parameter combination index')
plt.ylabel('CV mean accuracy')
plt.grid(True)
plt.show()


## 10. Evaluate best model and final interpretation

We show the confusion matrix, classification report, and final decision boundary (if 2D).


In [ ]:
best = grid.best_estimator_
y_pred_final = best.predict(X_test_final)
print('Final test accuracy:', accuracy_score(y_test_final, y_pred_final))
print('\nClassification report:\n')
print(classification_report(y_test_final, y_pred_final))
cm = confusion_matrix(y_test_final, y_pred_final)
ConfusionMatrixDisplay(cm).plot()
plt.title('Confusion Matrix - Final tuned model')
plt.show()

# If final data is 2D, plot decision boundary
if X_train_final.shape[1] == 2:
    plot_decision_boundary(best, X_train_final, y_train_final, title='Decision boundary - Final tuned RF')


## Key points and keywords (for your PDF)

- **Random Forest**: ensemble of decision trees, reduces variance by averaging.
- **max_depth**: deeper trees fit more complex patterns but risk overfitting.
- **n_estimators**: more trees generally increase stability; diminishing returns after a point.
- **max_features**: controls randomness and decorrelation among trees.
- **min_samples_leaf**: regularises trees by requiring minimum samples in leaves.
- **Permutation importance**: model-agnostic, measures drop in performance when a feature is shuffled.
- **PCA vs NCA**: PCA is unsupervised variance-based projection; NCA is supervised and often better for classification separation.

Include short interpretations under each plot in your PDF and relate observations back to bias–variance intuition.
